In [ ]:
import asyncio
import aiohttp

# Function to get distance and time for a single row using aiohttp
async def get_distance_and_time_async(session, home_lat, home_lon, office_lat, office_lon):
    url = f"http://router.project-osrm.org/route/v1/driving/{home_lon},{home_lat};{office_lon},{office_lat}?overview=false"
    
    try:
        async with session.get(url) as response:
            if response.status == 200:
                data = await response.json()
                distance = data['routes'][0]['legs'][0]['distance'] / 1000  # Convert meters to kilometers
                duration = data['routes'][0]['legs'][0]['duration'] / 60  # Convert seconds to minutes
                return distance, duration
            else:
                return None, None
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None, None

# Function to fetch distances and times concurrently with semaphore for concurrency control
async def get_distances_and_times_concurrently(df, max_concurrent_requests=10, batch_size=100):
    semaphore = asyncio.Semaphore(max_concurrent_requests)  # Limit concurrency
    async with aiohttp.ClientSession() as session:
        tasks = []
        
        # Create tasks for all rows in the DataFrame with semaphore
        for _, row in df.iterrows():
            tasks.append(get_distance_and_time_with_semaphore(semaphore, session, row['home_lat'], row['home_lon'], row['office_lat'], row['office_lon']))
        
        # To avoid too many tasks being created at once, we'll process them in smaller batches.
        results = []
        for i in range(0, len(tasks), batch_size):
            batch = tasks[i:i + batch_size]
            results.extend(await asyncio.gather(*batch))  # Gather the batch results
        
        return results

# Function to wrap the async request with semaphore
async def get_distance_and_time_with_semaphore(semaphore, session, home_lat, home_lon, office_lat, office_lon):
    async with semaphore:  # This ensures that only a limited number of requests are sent concurrently
        return await get_distance_and_time_async(session, home_lat, home_lon, office_lat, office_lon)

# Function to run the main async function
async def main():
    # Assuming df_extracted_700 is your DataFrame
    distances_times = await get_distances_and_times_concurrently(df_extracted_700, max_concurrent_requests=20, batch_size=100)
    
    # Filter out None values if any API calls failed
    distances_times = [result for result in distances_times if result != (None, None)]
    
    # Add the results to the DataFrame
    df_extracted_700['Distance'] = [result[0] for result in distances_times]
    df_extracted_700['Baseline_time'] = [result[1] for result in distances_times]
    
    # Display the updated dataframe
    print(df_extracted_700.head())

# Run the async main function inside an event loop
if __name__ == "__main__":
    await main()
